<a href="https://colab.research.google.com/github/thapaangel/Heuristic-021311/blob/main/Heuristic_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#8 Puzzle
from collections import deque
import copy

# Goal state
GOAL_STATE = [[1, 2, 3],
              [4, 5, 6],
              [7, 8, 0]]

# Directions with their coordinate change
MOVES = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

def manhattan_distance(state):
    """Calculate total Manhattan distance of the current state from the goal state."""
    distance = 0
    for i in range(3):
        for j in range(3):
            val = state[i][j]
            if val != 0:
                goal_x, goal_y = (val - 1) // 3, (val - 1) % 3
                distance += abs(goal_x - i) + abs(goal_y - j)
    return distance

def get_blank_position(state):
    """Find the position of the blank (0) tile."""
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j

def is_goal(state):
    """Check if the state is the goal state."""
    return state == GOAL_STATE

def state_to_tuple(state):
    """Convert 2D list state to a tuple for hashing in sets."""
    return tuple([num for row in state for num in row])

def generate_moves(state):
    """Generate all valid moves from the current state."""
    blank_i, blank_j = get_blank_position(state)
    possible_moves = []

    for move, (di, dj) in MOVES.items():
        new_i, new_j = blank_i + di, blank_j + dj
        if 0 <= new_i < 3 and 0 <= new_j < 3:
            new_state = copy.deepcopy(state)
            # Swap blank with the target tile
            new_state[blank_i][blank_j], new_state[new_i][new_j] = new_state[new_i][new_j], new_state[blank_i][blank_j]
            possible_moves.append((move, new_state))
    return possible_moves

def bfs(start_state):
    """Breadth-First Search to find the optimal path to the goal."""
    queue = deque()
    visited = set()

    queue.append((start_state, [], 0))
    visited.add(state_to_tuple(start_state))

    print("Initial State:")
    for row in start_state:
        print(row)
    print("Initial Manhattan Distance:", manhattan_distance(start_state))
    print()

    while queue:
        current_state, path, depth = queue.popleft()

        if is_goal(current_state):
            print("Goal reached!")
            return path

        for move, next_state in generate_moves(current_state):
            state_key = state_to_tuple(next_state)
            if state_key not in visited:
                visited.add(state_key)
                heuristic = manhattan_distance(next_state)
                print(f"Explored Move: {move}")
                for row in next_state:
                    print(row)
                print("Manhattan Distance:", heuristic)
                print()
                queue.append((next_state, path + [move], depth + 1))

    return None

# Example input
initial_state = [[1, 2, 3],
                 [4, 0, 5],
                 [7, 8, 6]]

solution_path = bfs(initial_state)

if solution_path:
    print("Optimal solution path (moves):")
    print(solution_path)
else:
    print("No solution found.")

Initial State:
[1, 2, 3]
[4, 0, 5]
[7, 8, 6]
Initial Manhattan Distance: 2

Explored Move: up
[1, 0, 3]
[4, 2, 5]
[7, 8, 6]
Manhattan Distance: 3

Explored Move: down
[1, 2, 3]
[4, 8, 5]
[7, 0, 6]
Manhattan Distance: 3

Explored Move: left
[1, 2, 3]
[0, 4, 5]
[7, 8, 6]
Manhattan Distance: 3

Explored Move: right
[1, 2, 3]
[4, 5, 0]
[7, 8, 6]
Manhattan Distance: 1

Explored Move: left
[0, 1, 3]
[4, 2, 5]
[7, 8, 6]
Manhattan Distance: 4

Explored Move: right
[1, 3, 0]
[4, 2, 5]
[7, 8, 6]
Manhattan Distance: 4

Explored Move: left
[1, 2, 3]
[4, 8, 5]
[0, 7, 6]
Manhattan Distance: 4

Explored Move: right
[1, 2, 3]
[4, 8, 5]
[7, 6, 0]
Manhattan Distance: 4

Explored Move: up
[0, 2, 3]
[1, 4, 5]
[7, 8, 6]
Manhattan Distance: 4

Explored Move: down
[1, 2, 3]
[7, 4, 5]
[0, 8, 6]
Manhattan Distance: 4

Explored Move: up
[1, 2, 0]
[4, 5, 3]
[7, 8, 6]
Manhattan Distance: 2

Explored Move: down
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
Manhattan Distance: 0

Explored Move: down
[4, 1, 3]
[0, 2, 5]
[7, 8, 6]
M

In [2]:
#Hill Climbing Block Arrangement
def heuristic(state, goal):
    """Heuristic: count how many blocks are out of place compared to the goal."""
    return sum(1 for i in range(len(state)) if state[i] != goal[i])

def get_neighbors(state):
    """Generate neighbors by swapping adjacent blocks."""
    neighbors = []
    for i in range(len(state) - 1):
        new_state = state.copy()
        # Swap adjacent blocks
        new_state[i], new_state[i+1] = new_state[i+1], new_state[i]
        neighbors.append((new_state, f"swap({i}, {i+1})"))
    return neighbors

def hill_climbing(start_state, goal_state):
    """Hill Climbing algorithm to solve the block arrangement problem."""
    current_state = start_state
    current_h = heuristic(current_state, goal_state)
    path = []

    print("Initial Stack:", current_state)
    print("Initial Heuristic:", current_h)
    print()

    while True:
        neighbors = get_neighbors(current_state)
        next_state = None
        next_h = current_h

        # Evaluate all neighbors to find the one with lowest heuristic
        for state, move in neighbors:
            h = heuristic(state, goal_state)
            print("Evaluating:", state, "| Heuristic:", h)
            if h < next_h:
                next_h = h
                next_state = (state, move)

        print()

        # No better neighbor found
        if next_state is None:
            print("Stuck at local minimum or reached goal.")
            break

        # Move to better neighbor
        current_state, move = next_state
        current_h = next_h
        path.append(move)
        print("Move:", move)
        print("Current Stack:", current_state)
        print("Current Heuristic:", current_h)
        print()

        # If goal reached
        if current_h == 0:
            break

    if current_h == 0:
        print("Goal reached!")
        print("Solution path:", path)
    else:
        print("Hill Climbing got stuck. Best state found:")
        print("Final Stack:", current_state)
        print("Path tried:", path)

# Example input
initial_stack = ['C', 'A', 'D', 'B']
goal_stack = ['A', 'B', 'C', 'D']

hill_climbing(initial_stack, goal_stack)

Initial Stack: ['C', 'A', 'D', 'B']
Initial Heuristic: 4

Evaluating: ['A', 'C', 'D', 'B'] | Heuristic: 3
Evaluating: ['C', 'D', 'A', 'B'] | Heuristic: 4
Evaluating: ['C', 'A', 'B', 'D'] | Heuristic: 3

Move: swap(0, 1)
Current Stack: ['A', 'C', 'D', 'B']
Current Heuristic: 3

Evaluating: ['C', 'A', 'D', 'B'] | Heuristic: 4
Evaluating: ['A', 'D', 'C', 'B'] | Heuristic: 2
Evaluating: ['A', 'C', 'B', 'D'] | Heuristic: 2

Move: swap(1, 2)
Current Stack: ['A', 'D', 'C', 'B']
Current Heuristic: 2

Evaluating: ['D', 'A', 'C', 'B'] | Heuristic: 3
Evaluating: ['A', 'C', 'D', 'B'] | Heuristic: 3
Evaluating: ['A', 'D', 'B', 'C'] | Heuristic: 3

Stuck at local minimum or reached goal.
Hill Climbing got stuck. Best state found:
Final Stack: ['A', 'D', 'C', 'B']
Path tried: ['swap(0, 1)', 'swap(1, 2)']
